In [1]:
!pwd
! source ../../../../st_env/bin/activate

/data1/alistvt/tes/multidoc-conv-qa/src/retriever/notebooks


In [2]:
import json
import pandas as pd

from simpletransformers.retrieval import RetrievalModel, RetrievalArgs

/data1/alistvt/tes/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'tensorboardX'

# Experiment 1: No history

In [9]:
def construct_retriever_dataset(dial_path, docs_path):
    "retriever_dataset[query_text | title | gold_passage]"
    retriever_dataset = []

    with open(dial_path, 'r') as f:
        questions_dataset = json.load(f)['dial_data']
    
    with open(docs_path, 'r') as f:
        docs_dataset = json.load(f)['doc_data']

    for domain, domain_dials in questions_dataset.items():
        for dial in domain_dials:
            for i, turn in enumerate(dial['turns'][:-1]):
                if turn['role'] == 'user':
                    if dial['turns'][i+1]['role'] == 'agent':
                        agent_turn = dial['turns'][i+1]
                        question = dial['turns'][i]['utterance']

                        if len(agent_turn['references']):
                            reference = agent_turn['references'][0]
                            doc_id = reference['doc_id']
                            span_id = reference['id_sp']

                            retriever_dataset.append({
                                'query_text': question,
                                'title': domain + ' <SEP> ' + docs_dataset[doc_id]['title'],
                                'gold_passage': docs_dataset[doc_id]['doc_text']
                            })
                        else:
                            # TODO: unknown
                            pass
                    else:
                        continue
    return retriever_dataset


In [10]:
train_data = construct_retriever_dataset(
    '../../../dataset/multidoc2dial/v1.0/multidoc2dial_dial_train.json',
    '../../../dataset/multidoc2dial/v1.0/multidoc2dial_doc.json'
)

eval_data = construct_retriever_dataset(
    '../../../dataset/multidoc2dial/v1.0/multidoc2dial_dial_validation.json',
    '../../../dataset/multidoc2dial/v1.0/multidoc2dial_doc.json'
)

train_df = pd.DataFrame(train_data)
eval_data = pd.DataFrame(eval_data)


NameError: name 'json' is not defined